# Top Airlines Over Time

This notebook calculates and exports stacked bar charts of:

- Top airlines by month
- Top airlines by year

In [16]:
%run ../pathutils.ipynb
%run ../definitions.ipynb
%run ../database.ipynb
%run ../export.ipynb
%run utils.ipynb

In [17]:
# Top "N" airlines to include
top_n = 10

In [18]:
import matplotlib.pyplot as plt
import seaborn as sns

def export_monthly_chart(export_folder_path, data):
    fig, ax = plt.subplots(figsize=(16, 7))

    data.plot(
        kind="bar",
        stacked=True,
        figsize=(12, 6),
        colormap="tab20",
        ax = ax
    )

    # De-clutter the X-axis labels by showing only every other one
    # xticks = ax.get_xticks()
    xticklabels = [label.get_text() for label in ax.get_xticklabels()]
    new_labels = [label if i % 2 == 0 else "" for i, label in enumerate(xticklabels)]
    ax.set_xticklabels(new_labels, rotation=45)

    # Add titles, labels and legends
    plt.title("Top Airlines by Sightings Per Month (Stacked Bar Chart)")
    plt.ylabel("Number of Sightings")
    plt.xlabel("Month")
    plt.xticks(rotation=90)
    plt.legend(title="Airline", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()

    # Export the chart
    export_chart(export_folder_path, f"top-airlines-over-time-monthly", "png")

    # Close the plot
    plt.close(fig)

In [19]:
import matplotlib.pyplot as plt
import seaborn as sns

def export_yearly_chart(export_folder_path, data):
    fig, ax = plt.subplots(figsize=(12, 6))

    data.plot(
        kind="bar",
        stacked=True,
        figsize=(12, 6),
        colormap="tab20",
        ax = ax
    )

    # Add titles, labels and legends
    plt.title("Top Airlines by Sightings Per Year")
    plt.ylabel("Number of Sightings")
    plt.xlabel("Year")
    plt.xticks(rotation=0)
    plt.legend(title="Airline", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()

    # Export the chart
    export_chart(export_folder_path, f"top-airlines-over-time-yearly", "png")

    # Close the plot
    plt.close(fig)

In [ ]:
import warnings

# Construct the query
query = construct_query("aircraft", "sightings.sql", {})

try:
    # Run the query to load the data - this raises a ValueError if there's no data
    df = query_data("aircraft", query)

    # Create Year and Month columns
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.to_period("M")

    # Aggregate by year, determine the top 'N' airlines and filter so only they remain
    airlines_by_year = df.groupby(["Year", "Airline"]).size().reset_index(name="Sightings")
    top_n_airlines = airlines_by_year.groupby("Airline")["Sightings"].sum().nlargest(top_n).index
    top_yearly = airlines_by_year[airlines_by_year["Airline"].isin(top_n_airlines)]

    # Aggregate by month and filter to leave only the top 'N' airlines
    airlines_by_month = df.groupby(["Month", "Airline"]).size().reset_index(name="Sightings")
    top_monthly = airlines_by_month[airlines_by_month["Airline"].isin(top_n_airlines)]

    # Convert 'month' Period to datetime for plotting
    warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
    top_monthly["Month"] = top_monthly["Month"].dt.to_timestamp()

    # Pivot the yearly data to allow a stacked bar chart plot:
    # rows = year, columns = airline
    pivoted_yearly = top_yearly.pivot(index="Year", columns="Airline", values="Sightings").fillna(0)

    # Pivot the monthly data to allow a stacked bar chart plot:
    # rows = month, columns = airline, values = sightings
    # Ensure sorting by month after pivoting
    pivoted_monthly = top_monthly.pivot(index="Month", columns="Airline", values="Sightings").fillna(0)
    pivoted_monthly = pivoted_monthly.sort_index()

    # Export the data to excel
    export_folder_path = get_export_folder_path()
    export_to_spreadsheet(export_folder_path, "top-airlines-over-time-monthly.xlsx", {
        "Monthly": top_monthly
    })
    export_to_spreadsheet(export_folder_path, "top-airlines-over-time-yearly.xlsx", {
        "Yearly": top_yearly,
    })

    # Export the charts
    export_monthly_chart(export_folder_path, pivoted_monthly)
    export_yearly_chart(export_folder_path, pivoted_yearly)

except ValueError:
    print(f"WARNING: No data found")